# **FinalNotebook**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
df = pd.read_csv("/kaggle/input/sentiment-prediction-on-movie-reviews-term-break1/train.csv (1)/train.csv")

In [ ]:
df.head()

In [ ]:
df_movies = pd.read_csv("/kaggle/input/sentiment-prediction-on-movie-reviews-term-break1/movies.csv (1)/movies.csv")

In [ ]:
df_movies.shape

In [ ]:
df_movies.head()

In [ ]:
df_movies.isna().sum()

I have not used any features from the meta data as there are many empties and Imputing them might lead to data bias. Although some of it could be usable.

In [ ]:
df_movies.info()

In [ ]:
df_movies.shape

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
import seaborn as sns
sns.countplot(data=df , x = "sentiment")

In [ ]:
sns.countplot(data=df , x = "isFrequentReviewer")

In [ ]:
import numpy as np
aud_score = list(df_movies['audienceScore'].dropna())
# print(aud_score)
print(f"Mean of Audience score : {sum(aud_score)/len(aud_score)}")
sns.histplot(data=aud_score)

In [ ]:
df.isna().sum()

In [ ]:
print(df["reviewText"][8])
print(df["sentiment"][8])

In [ ]:
df.head()

In [ ]:
test_set = pd.read_csv("/kaggle/input/sentiment-prediction-on-movie-reviews-term-break1/test.csv (1)/test.csv")

In [ ]:
test_set.head()

In [ ]:
test_set.columns

In [ ]:
df.isna().sum()

**Data Cleaning and Feature Selection**

In [ ]:
df = df.dropna()
# X = df["reviewText"]
y = df["sentiment"]
X = df.drop(columns = ["movieid" , "reviewerName" , "sentiment" ])

**Label Encoding target variable and feature encoding**

In [ ]:
from sklearn import preprocessing
lb_target = preprocessing.LabelBinarizer()
y = lb_target.fit_transform(y)

In [ ]:
y[0] , df['sentiment'][0]

**POSTIVE - 1 , NEGATIVE - 0**

In [ ]:
from sklearn import preprocessing
lb_feature = preprocessing.LabelBinarizer()
X["isFrequentReviewer"] = lb_feature.fit_transform(X["isFrequentReviewer"])

In [ ]:
X.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

In [ ]:
X_train.shape , y_train.shape

# **Model Training**
1. Converting the text features into numerical data using TF-IDF.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

model = LogisticRegression(max_iter = 500)
vectorizer1 = TfidfVectorizer()
# lb = preprocessing.LabelBinarizer()

# construct the column transfomer
column_transformer = ColumnTransformer(
    [('tfidf1', vectorizer1, 'reviewText') ] ,
#     ('lb', lb, 'isFrequentReviewer')] ,
    remainder = 'passthrough')

# fit the model
pipe = Pipeline([
                  ('feature_ext', column_transformer),
                  ('classify', model)
                ])
pipe.fit(X_train,y_train)

# **Model Testing**

In [ ]:
pipe.score(X_test,y_test)

# **Logistic Regression with Cross Validation and Hyper-parameter tuning.**

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

k_folds = KFold(n_splits = 5)

scores = cross_val_score(pipe, X_train, y_train, cv = k_folds)

print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix 

logisticRegr = LogisticRegression()

param_grid = {'classify__C': [0.1, 1, 10, 100],  
              'classify__penalty' : ['l1', 'l2', 'elasticnet', None], 
              'classify__tol':[0.0001 , 0.001 , 0.01 , 0.1]
              } 

best_logistic_model = GridSearchCV(pipe, param_grid, refit = True)

best_logistic_model.fit(X_train, y_train) 

print(classification_report(y_test, best_logistic_model.predict(X_test))) 

In [ ]:
print(best_logistic_model.best_params_) 

**best params : {'classify__C': 1, 'classify__penalty': 'l2', 'classify__tol': 0.0001}**

In [ ]:
print(classification_report(y_test, best_logistic_model.predict(X_test))) 

In [ ]:
test_set = pd.read_csv("/kaggle/input/sentiment-prediction-on-movie-reviews-term-break1/test.csv (1)/test.csv")

In [ ]:
test_set.isna().sum()

In [ ]:
test_set.head()

**Preparing test set for prediction.**

In [ ]:
test_set["isFrequentReviewer"] = lb_feature.transform(test_set["isTopCritic"])
test_set['reviewText'] = test_set['reviewText'].fillna('')
test_set = test_set.drop(columns = ['movieid' , 'reviewerName' , 'isTopCritic'])

In [ ]:
test_set.head()

In [ ]:
X_pred = best_logistic_model.predict(test_set)

In [ ]:
X_pred.shape

In [ ]:
final_sub = pd.DataFrame()
final_sub['id'] = [x for x in range(len(X_pred))]
final_sub['sentiment'] = ["POSITIVE" if label == 1 else "NEGATIVE" for label in X_pred]

In [ ]:
final_sub['sentiment'].value_counts()

In [ ]:
final_sub.to_csv("submission.csv",index=False)

# **Support Vector Machine - Classifier**

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer

model = SVC(max_iter = 500)
vectorizer1 = TfidfVectorizer()
# lb = preprocessing.LabelBinarizer()

# construct the column transfomer
column_transformer = ColumnTransformer(
    [('tfidf1', vectorizer1, 'reviewText') ] ,
#     ('lb', lb, 'isFrequentReviewer')] ,
    remainder = 'passthrough')

# fit the model
pipe_1 = Pipeline([
                  ('feature_ext', column_transformer),
                  ('classify', model)
                ])
pipe_1.fit(X_train,y_train)

In [ ]:
pipe_1.score(X_test,y_test)

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

k_folds = KFold(n_splits = 5)

scores = cross_val_score(pipe_1, X_train, y_train, cv = k_folds)

print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix 

param_grid = {'classify__C': [0.1, 1, 10, 100],  
              'classify__kernel' : ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'], 
#               'classify__tol':[0.0001 , 0.001 , 0.01 , 0.1],
#               'classify__degree':[1 , 2 , 3 , 4]
              } 

best_logistic_model = GridSearchCV(pipe_1, param_grid, refit = True)

best_logistic_model.fit(X_train, y_train) 

print(classification_report(y_test, best_logistic_model.predict(X_test))) 

# **Naive Bayes**

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

model = MultinomialNB()
vectorizer1 = TfidfVectorizer()
# lb = preprocessing.LabelBinarizer()

# construct the column transfomer
column_transformer = ColumnTransformer(
    [('tfidf1', vectorizer1, 'reviewText') ] ,
#     ('lb', lb, 'isFrequentReviewer')] ,
    remainder = 'passthrough')

# fit the model
pipe_2 = Pipeline([
                  ('feature_ext', column_transformer),
                  ('classify', model)
                ])
pipe_2.fit(X_train,y_train)

In [ ]:
pipe_2.score(X_test,y_test)

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

k_folds = KFold(n_splits = 5)

scores = cross_val_score(pipe_2, X_train, y_train, cv = k_folds)

print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix 

param_grid = {'classify__alpha': [0.1, 1, 10, 100],  
              'classify__force_alpha' : [True , False]
              } 

best_logistic_model = GridSearchCV(pipe_2, param_grid, refit = True)

best_logistic_model.fit(X_train, y_train) 

print(classification_report(y_test, best_logistic_model.predict(X_test))) 